In [1]:
import numpy as np
from sklearn.datasets import fetch_openml
import torch

boston = fetch_openml(name='boston', version=1)

X = boston.data.to_numpy().astype(float)
Y = boston.target.to_numpy().astype(float)

np.random.seed(123)
torch.random.manual_seed(123)

idx = np.random.permutation(len(X))

X = X[idx]
Y = Y[idx]

X_training = torch.Tensor(X[50:])
Y_training = torch.Tensor(Y[50:])

X_testing = torch.Tensor(X[:50])
Y_testing = torch.Tensor(Y[:50])

In [2]:
from torch import nn

class LinearRegression(nn.Module):
    def __init__(self, in_size, out_size):
        self.linear = nn.Linear(in_size, out_size)
    
    def forward(self,x):
        result = self.linear(x)
        return result

In [4]:
from torch.utils.tensorboard import SummaryWriter

lr = 0.01
epochs = 300

writer = SummaryWriter('logs/linear_regressor_1')

means = X_training.mean(0)
stds = X_training.std(0)

X_training_norm = (X_training-means)/stds
X_testing_norm = (X_testing-means)/stds

reg = LinearRegressor(13,1)
criterion = nn.MSELoss()

optimizer=torch.optim.SGD(reg.parameters(),lr=lr)

for e in range(epochs):
    reg.train()
    output = reg(X_training_norm)
    l = criterion(output.view(-1), Y_training)

    writer.add_scalar('loss/train', l.item(), global_step=e)

    l.backward()

    optimizer.step()
    optimizer.zero_grad()

    reg.eval()

    with torch.set_grad_enabled(False):
        y_test = reg(X_testing_norm)
        l = criterion(y_test.view(-1), Y_testing)
        writer.add_scalar('loss/test', l.item(), global_step=e)


FailedPreconditionError: logs/linear_regressor_1 is not a directory